# Procesamiento del Lenguaje Natural

# Trabajo Práctico Final

# Vercesi Patricio

# Ejercicio 1

## Estructura del coso
1. csv
2. grafo
3. docx
   1. limpieza + splitting???
   2. embedding
   3. vectorizar
   4. algo mas?
4. entrenar LLM?
   4. poder pasarle prompt más como usuario?
5. algo más que estaba en la imagen pero me olvidé

## Orden a programarlo
6. entrenamiento
7. prediccion
8. otras weas que estaban en la imagen
9. limpieza de docx y splitting  langchain.recursivecharactertextsplitter?
10. embeddizar docx (100 págs) (con OpenAI por ej)
11. vectorizarlo (en chromaDB)
12. hacerle algo mas (??) (primero darlo solo como contexto y desp reentrenar el clasificador)

Extra?
1. analisis exploratorio
2. cambiar a file upload method
3. sacar los import e install innecesarios

## Introducción

Elegí de tema los videojuegos y saqué la información de la página en español de Wikipedia al respecto y muchas otras páginas de Wikipedia vinculadas a la misma.

## Importación de librerías

In [1]:
#Importación de librerias
from google.colab import files, drive
import pandas as pd
import openpyxl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

## Langchain
!pip install -q -U langchain-google-genai
from langchain_google_genai import ChatGoogleGenerativeAI


import gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.9 MB/s eta 0:00:00


In [2]:
!pip install python-decouple jinja2 -q

import requests
import json
from decouple import config
from jinja2 import Template
from google.colab import userdata

In [3]:
!pip install cohere -q
!pip install tiktoken -q
!pip install openai -q
import cohere
!pip install llama_index --upgrade -q
!pip install sentence-transformers pypdf langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 2.3 MB/s eta 0:00:00
  

In [4]:
%%writefile /usr/local/lib/python3.10/dist-packages/openai/_utils/_streams.py
from typing import Any
from typing_extensions import AsyncIterator
from typing import Iterator # import Iterator from the correct library

def consume_sync_iterator(iterator: Iterator[Any]) -> None:
    for _ in iterator:
        ...

async def consume_async_iterator(iterator: AsyncIterator[Any]) -> None:
    async for _ in iterator:
        ...

Overwriting /usr/local/lib/python3.10/dist-packages/openai/_utils/_streams.py


In [5]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
from llama_index import ServiceContext
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from jinja2 import Template
import requests
from decouple import config

## Importación de datos

In [6]:
# Link con archivos
url = 'https://drive.google.com/drive/folders/1Y3iEEzuQ9jZrgFIFbO7klIZkAeId6ZHL?usp=drive_link'
# Descarga carpeta
gdown.download_folder(url, quiet=True, output='data')
file_path = 'data/'

### CSV

In [7]:
df = pd.read_csv(file_path+'consolas_mejor_vendidas.csv', encoding='latin1')
df

,Plataforma,Tipo,Empresa,Lanzamiento,Unidades vendidas
0,PlayStation 2,Domestica,Sony,2000,>155 millones
1,Nintendo DS,Portatil,Nintendo,2004,"154,02 millones"
2,Nintendo Switch,Hibrida,Nintendo,2017,"132,46 millones"
3,Game Boy y Game Boy Color,Portatil,Nintendo,"1989, 1998","118,69 millones"
4,PlayStation 4,Domestica,Sony,2013,"117,2 millones"
5,PlayStation,Domestica,Sony,1994,"102,49 millones"
6,Wii,Domestica,Nintendo,2006,"101,63 millones"
7,PlayStation 3,Domestica,Sony,2006,">87,4 millones"
8,Xbox 360,Domestica,Microsoft,2005,84 millones
9,Game Boy Advance,Portatil,Nintendo,2001,"81,5 millones"


### Docx/ PDF

guardar los textos en uno o varios .docx para desp leer

### Base de datos de grafos

Bajé un CSV de la página https://query.wikidata.org que te deja hacer queries a Wikidata que es una base de datos de grafos online.

Esta fue la query que hice:

In [8]:
"""
SELECT ?GameLabel (MIN(?Date) AS ?InitialReleaseDate) (GROUP_CONCAT(DISTINCT ?GenreLabel; separator=", ") AS ?Genres)
WHERE {
  ?Game wdt:P178 wd:Q193559 ;
        wdt:P31 wd:Q7889 ;
        rdfs:label ?GameLabel ;
        wdt:P577 ?Date ;
        wdt:P136 ?Genre .
  ?Genre rdfs:label ?GenreLabel .
  FILTER ((LANG(?GameLabel) = "en") && (LANG(?GenreLabel) = "en"))
}
GROUP BY ?GameLabel
"""

'\nSELECT ?GameLabel (MIN(?Date) AS ?InitialReleaseDate) (GROUP_CONCAT(DISTINCT ?GenreLabel; separator=", ") AS ?Genres)\nWHERE {\n  ?Game wdt:P178 wd:Q193559 ;\n        wdt:P31 wd:Q7889 ;\n        rdfs:label ?GameLabel ;\n        wdt:P577 ?Date ;\n        wdt:P136 ?Genre .\n  ?Genre rdfs:label ?GenreLabel .\n  FILTER ((LANG(?GameLabel) = "en") && (LANG(?GenreLabel) = "en"))\n}\nGROUP BY ?GameLabel\n'

In [9]:
df = pd.read_csv(file_path+'valve_games_with_dates.csv', encoding='latin1')
df

,GameLabel,InitialReleaseDate,Genres
0,Left 4 Dead 2,2009-11-17T00:00:00Z,"first-person shooter, survival horror, post-ap..."
1,Half-Life 2: Episode One,2006-06-01T00:00:00Z,"first-person shooter, science fiction video game"
2,Counter-Strike: Global Offensive,2012-08-21T00:00:00Z,"first-person shooter, action game"
3,Day of Defeat: Source,2005-09-26T00:00:00Z,first-person shooter
4,Counter-Strike Neo,2005-10-27T00:00:00Z,"first-person shooter, arcade video game machine"
5,Half-Life 2: Episode Two,2007-10-10T00:00:00Z,"first-person shooter, science fiction video game"
6,Dota 2,2013-07-09T00:00:00Z,"real-time strategy, action game, multiplayer o..."
7,Half-Life 2: Lost Coast,2005-10-27T00:00:00Z,first-person shooter
8,Counter-Strike: Source,2004-11-01T00:00:00Z,"first-person shooter, tactical shooter"
9,Portal,2007-10-10T00:00:00Z,"science fiction, puzzle video game, first-pers..."


## Preparación de funciones y demás para usar la LLM

In [10]:
# Preparamos el retriever de documentos
# Construimos un índice de documentos a partir de los datos de la carpeta
reader = SimpleDirectoryReader("data")
documents = reader.load_data()

# Embedding de todos los documentos
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2'))
index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=ServiceContext.from_defaults(embed_model=embed_model, llm=None))

# Construimos un retriever a partir del índice, para realizar la búsqueda vectorial de documentos
retriever = index.as_retriever(similarity_top_k=1) # devuelve el documento más relevante

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


Parsing nodes:   0%|          | 0/520 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/527 [00:00<?, ?it/s]

In [14]:
# Todo el procesamiento para convertir a las prompts en lo que le queremos pasar a la LLM
def prompt_processing(queries, nodes_lists = False):
    # Añadimos el contexto de la pregunta
    contexted_queries = []

    for i in range(len(queries)):
        contexted_query = ''
        # si hay informacion contextual la agregamos
        if nodes_lists:
            # pasamos el contexto de nodos a string
            context = ''
            for node in nodes_lists[i]:
                context += f"{node.text}\n"
            # lo agregamos
            contexted_query += "La información contextual es:\n"
            contexted_query += "---------------------\n"
            contexted_query += f"{context}\n"
            contexted_query += "---------------------\n"
        # y en cualquier caso siempre lo contextuamos con el formato QA
        contexted_query += "Pregunta: \n"
        contexted_query += f"{queries[i]}\n"
        contexted_query += "Respuesta: \n"

        contexted_queries.append(contexted_query)

    # agregamos msjs del sistema
    messages_list = []

    for contexted_query in contexted_queries:
        messages = [
            {
                "role": "system",
                "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y usando el contexto que te brindan.",
            },
            {
                "role": "user",
                "content": contexted_query
            }
        ]

        messages_list.append(messages)

    # Conversión de mensajes del sistema o del usuario a formato Zephyr
    template_str = "{% for message in messages %}"
    template_str += "{% if message['role'] == 'user' %}"
    template_str += "<|user|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'assistant' %}"
    template_str += "<|assistant|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'system' %}"
    template_str += "<|system|>{{ message['content'] }}</s>\n"
    template_str += "{% else %}"
    template_str += "<|unknown|>{{ message['content'] }}</s>\n"
    template_str += "{% endif %}"
    template_str += "{% endfor %}"
    template_str += "{% if add_generation_prompt %}"
    template_str += "<|assistant|>\n"
    template_str += "{% endif %}"
    # Crear un objeto de plantilla con la cadena de plantilla
    template = Template(template_str)

    final_prompts = []

    for messages in messages_list:
        # Renderizar la plantilla con los mensajes proporcionados
        formatted_prompt = template.render(messages=messages, add_generation_prompt=True)
        final_prompts.append(formatted_prompt)

    return final_prompts

In [15]:
# Aquí hacemos la llamada al modelo
def generate_answers(prompts: str, max_new_tokens: int = 768) -> None:
    try:
        # URL de la API de Hugging Face para la generación de texto
        zephyr_api = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        answers = []

        for prompt in prompts:
            # Datos para enviar en la solicitud POST
            # Sobre los parámetros: https://huggingface.co/docs/transformers/main_classes/text_generation
            data = {
                "inputs": prompt,
                "parameters": {
                    "max_new_tokens": max_new_tokens,
                    "temperature": 0.1,
                    "top_k": 50,
                    "top_p": 0.95
                }
            }

            # Realizamos la solicitud POST a la LLM
            response = requests.post(zephyr_api, json=data)
            # Extraer respuesta
            answer = response.json()[0]["generated_text"][len(prompt):]
            answers.append(answer)

        return answers

    except Exception as e:
        print(f"An error occurred: {e}")

In [19]:
def prompts2responses(queries, context = True):
    # contextos de las queries
    nodes_lists = []
    if context:
        for query in queries:
            nodes_lists.append(retriever.retrieve(query))

    # añadimos el contexto al prompt y lo procesamos
    final_prompts = prompt_processing(queries, nodes_lists)

    # obtenemos las respuestas
    answers = generate_answers(final_prompts)

    #imprimimos
    for i in range(len(queries)):
        print(
            "____________________________________________________________________\n\n"
            "# Pregunta:\n"
            f"{queries[i]}\n\n"
            "# Respuesta:\n"
            f"{answers[i]}\n\n"
        )
        if context:
            print(f"# Información sobre el documento usado:\n{nodes_lists[i][0]}\n")


In [17]:
default_queries = [
    '¿Cuáles son las 5 consolas mejor vendidas de la historia?',    #PS2, DS, Switch, Game Boy, PS
    '¿Qué año salió Half-Life 2: Lost Coast?',                      #2005
    '¿Cuántos capítulos tiene el libro "The Art of Game Design"?',  #33
]

## LLM sin información contextual

In [18]:
prompts2responses(default_queries, context = False)

____________________________________________________________________
Pregunta:
¿Cuáles son las 5 consolas mejor vendidas de la historia?
Respuesta:
1. PlayStation 2: Lanzada por Sony en 2000, se estiman más de 155 millones de unidades vendidas en todo el mundo, lo que la convierte en la consola más vendida de la historia.

2. PlayStation (original): La primera consola de Sony, lanzada en 1994, vendió más de 102 millones de unidades en todo el mundo.

3. Nintendo Entertainment System (NES): Lanzada por Nintendo en 1983, se estiman más de 61,91 millones de unidades vendidas en todo el mundo.

4. Super Nintendo Entertainment System (SNES): Lanzada por Nintendo en 1990, se estiman más de 49,10 millones de unidades vendidas en todo el mundo.

5. PlayStation 4: La última consola en la lista, lanzada por Sony en 2013, ha vendido más de 110 millones de unidades en todo el mundo hasta el momento.

Es importante mencionar que algunas fuentes consideran la Game Boy de Nintendo como la quinta cons

## LLM con información contextual

In [20]:
prompts2responses(default_queries, context = True)

____________________________________________________________________

# Pregunta:
¿Cuáles son las 5 consolas mejor vendidas de la historia?
# Respuesta:
Según las cifras de ventas proporcionadas, las 5 consolas mejor vendidas de la historia son:

1. PlayStation 2, lanzada en 2000 por Sony, con más de 155 millones de unidades vendidas.
2. Nintendo DS, lanzada en 2004, con 154,02 millones de unidades vendidas.
3. Nintendo Switch, lanzada en 2017, con 132,46 millones de unidades vendidas.
4. Game Boy y Game Boy Color, lanzados entre 1989 y 1998 por Nintendo, con 118,69 millones de unidades vendidas en total.
5. PlayStation, lanzada en 1994 por Sony, con 102,49 millones de unidades vendidas.

Es importante mencionar que las cifras de ventas pueden estar sujetas a cambios y actualizaciones, especialmente en el caso de consolas más recientes como la Nintendo Switch y la PlayStation 5, que aún están en venta.

Info del documento elegido:
Node ID: acd1b6b1-170b-4b4f-88fd-4852c0634588
Text: Pla

## Main (?

In [ ]:
#!pip install langchain

In [ ]:
"""
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Cargar el tokenizer, el retriever y el generador RAG pre-entrenado
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
retriever = RagRetriever.from_pretrained("facebook/rag-token-base")
generator = RagSequenceForGeneration.from_pretrained("facebook/rag-token-base")

# Función para obtener la respuesta
def get_answer(question):
    # Obtener pasajes relevantes
    retriever_output = retriever(question, return_tensors="pt")
    retrieved_passages = retriever_output["retrieved_passages"]

    # Generar la respuesta basada en los pasajes recuperados
    generated = generator(input_ids=retrieved_passages, labels=retrieved_passages)
    decoded = tokenizer.decode(generated[0], skip_special_tokens=True)

    return decoded

# Ejemplo de conversación
while True:
    user_input = input("Usuario: ")
    if user_input.lower() == 'salir':
        break

    # Obtener la respuesta basada en la entrada del usuario
    answer = get_answer(user_input)
    print("Bot:", answer)
"""

## Testear con otros prompts

In [ ]:
queries = [
    '¿De qué habla el libro "The Art of Game Design"?'
]

prompts2responses(queries, context = True)

## Conclusiones